https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
https://github.com/DeepLearningSandbox/DeepLearningSandbox/blob/master/transfer_learning/fine-tune.py

In [6]:
import os
import glob

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import keras.callbacks
from keras.optimizers import SGD

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
train_dir='../data_simple'
val_dir='../validation_simple'
nb_epoch=2
batch_size=32

IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3

FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 249

In [8]:
def get_nb_files(directory):
  """Get number of files by searching directory recursively"""
  if not os.path.exists(directory):
    return 0
  cnt = 0
  for r, dirs, files in os.walk(directory):
    for dr in dirs:
      cnt += len(glob.glob(os.path.join(r, dr + "/*")))
  return cnt

In [9]:
nb_train_samples = get_nb_files(train_dir)
nb_val_samples = get_nb_files(val_dir)
nb_classes = len(glob.glob(val_dir + "/*"))

print("Loaded %d training images, %d validation images, spanning over %d classes."%(nb_train_samples, nb_val_samples, nb_classes))

Loaded 1295 training images, 322 validation images, spanning over 2 classes.


In [10]:
model = InceptionV3(weights='imagenet', include_top=True)

In [11]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# data prep
train_datagen =  ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
)

validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
)

Found 1294 images belonging to 2 classes.
Found 321 images belonging to 2 classes.


## Data preprocessing

At this point, the top layers are well trained and we can start fine-tuning convolutional layers from inception V3. We will freeze the bottom N layers and train the remaining top layers.

let's visualize layer names and layer indices to see how many layers we should freeze:

In [12]:
for i, layer in enumerate(model.layers):
   print(i, layer.name) 

(0, 'input_2')
(1, 'conv2d_26')
(2, 'batch_normalization_25')
(3, 'activation_25')
(4, 'conv2d_27')
(5, 'batch_normalization_26')
(6, 'activation_26')
(7, 'conv2d_28')
(8, 'batch_normalization_27')
(9, 'activation_27')
(10, 'max_pooling2d_3')
(11, 'conv2d_29')
(12, 'batch_normalization_28')
(13, 'activation_28')
(14, 'conv2d_30')
(15, 'batch_normalization_29')
(16, 'activation_29')
(17, 'max_pooling2d_4')
(18, 'conv2d_34')
(19, 'batch_normalization_33')
(20, 'activation_33')
(21, 'conv2d_32')
(22, 'conv2d_35')
(23, 'batch_normalization_31')
(24, 'batch_normalization_34')
(25, 'activation_31')
(26, 'activation_34')
(27, 'average_pooling2d_3')
(28, 'conv2d_31')
(29, 'conv2d_33')
(30, 'conv2d_36')
(31, 'conv2d_37')
(32, 'batch_normalization_30')
(33, 'batch_normalization_32')
(34, 'batch_normalization_35')
(35, 'batch_normalization_36')
(36, 'activation_30')
(37, 'activation_32')
(38, 'activation_35')
(39, 'activation_36')
(40, 'mixed0')
(41, 'conv2d_41')
(42, 'batch_normalization_40')
(4

we chose to train the top 2 inception blocks, i.e. we will freeze the first 249 layers and unfreeze the rest:

In [13]:
for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
   layer.trainable = False
for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
   layer.trainable = True

we need to recompile the model for these modifications to take effect
we use SGD with a low learning rate

In [14]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

we train our model again (this time fine-tuning the top 2 inception blocks alongside the top Dense layers

In [15]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph-fine', histogram_freq=0, write_graph=True, write_images=True)

In [18]:
history_ft = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=nb_epoch,
    validation_data=validation_generator,
    validation_steps=nb_val_samples//batch_size,
    class_weight='auto',
    callbacks=[tbCallBack]
  )

Epoch 1/2


ValueError: Error when checking target: expected predictions to have shape (None, 1000) but got array with shape (32, 2)

Save the model

In [ ]:
model.save('disdat-keras-refine-simple.model')